In [1]:
from transformers import AutoTokenizer, ElectraForSequenceClassification
import torch

device = torch.device("cpu")

# 모델과 토크나이저 불러오기
num_classes = 6  # 클래스 수에 맞게 설정
model = ElectraForSequenceClassification.from_pretrained("monologg/koelectra-base-v3-discriminator", num_labels=num_classes)
tokenizer = AutoTokenizer.from_pretrained("monologg/koelectra-base-v3-discriminator")

# 저장된 모델 불러오기
model.load_state_dict(torch.load("modelfinal.pt"))

Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at monologg/koelectra-base-v3-discriminator and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


<All keys matched successfully>

In [2]:
class_names = ["중립", "슬픔", "분노", "불안", "행복", "당황"]
def predict_with_prob(input_sentence):
    # 입력 문장을 토큰화하고 모델 입력 형식으로 변환
    inputs = tokenizer(input_sentence, return_tensors="pt", max_length=75, padding=True, truncation=True)
    input_ids = inputs["input_ids"].to(device)
    attention_mask = inputs["attention_mask"].to(device)
    
    # 모델을 사용하여 감정 예측
    outputs = model(input_ids, attention_mask=attention_mask)
    logits = outputs.logits
    
    # 예측된 감정
    predicted_class_idx = torch.argmax(logits, dim=1).item()
    predicted_emotion = class_names[predicted_class_idx]

    # 소프트맥스 함수를 사용하여 확률 계산
    probs = torch.softmax(logits, dim=1)
    
    # 각 클래스(감정)별 확률을 리스트로 변환
    class_probs = probs.squeeze().tolist()

    # 결과 반환
    return predicted_emotion, class_probs


In [3]:
# 질문 무한반복하기! 0 입력시 종료
end = 1
while end == 1:
    sentence = input("하고싶은 말을 입력해주세요 : ")
    if sentence == "0":
        break
    print("입력한 문장:", sentence)
    
    # 감정 예측 및 각 클래스(감정)별 확률 출력
    predicted_emotion, class_probs = predict_with_prob(sentence)
    print("예측된 감정:", predicted_emotion)
    print("각 클래스별 확률:")
    for class_name, prob in zip(class_names, class_probs):
        print(f"{class_name}: {prob*100:.2f}%")
    print("\n")

입력한 문장: 학교가기 싫은데 친구들은 만나러 가고 싶어
예측된 감정: 불안
각 클래스별 확률:
중립: 0.06%
슬픔: 2.86%
분노: 2.36%
불안: 92.09%
행복: 0.83%
당황: 1.80%


입력한 문장: 과제가 너무 많아서 화가나
예측된 감정: 분노
각 클래스별 확률:
중립: 0.18%
슬픔: 0.89%
분노: 97.82%
불안: 0.40%
행복: 0.04%
당황: 0.66%


입력한 문장: 미안해 앞으로 잘할게
예측된 감정: 슬픔
각 클래스별 확률:
중립: 0.75%
슬픔: 97.47%
분노: 0.62%
불안: 0.37%
행복: 0.10%
당황: 0.69%


